# Testing DRPredICT validation

**Authorship:**
Adam Klie, *03/29/2022*
***
**Description:**
Notebook for testing the validation of DRPredICT architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li>Test with drug embedding</li></b>
    <b><li>Set-up full test in this notebook</li></b>
    <b><li>Add command line functionality</li></b>
    <b><li>Create configs for each run</li></b>
    <b><li>Submit jobs to GPU</li></b>
    </ul>
</div>

In [1]:
import numpy as np
import pandas as pd
import torch

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import sys
sys.path.append("../drpredict")

# Random dataset
<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of our eugene architectures</b>
</div>

## Instantiate DRPredICT architecture: VanillaAE

In [38]:
from VanillaAE import VanillaAE

In [39]:
encoder_args = {"activation": "sigmoid"}
decoder_args = {"dropout_rate": 0.2}

In [40]:
mutAE = VanillaAE(1000, 10, hidden_dims=[500], encoder_kwargs=encoder_args, decoder_kwargs=decoder_args)

In [41]:
x = torch.randn(10, 1000)
out = mutAE(x)
out.shape

torch.Size([10, 1000])

## Load data

In [8]:
from torch.utils.data import dataset
from torch.utils.data import DataLoader

### Training set

In [9]:
training_dataset = dataset.TensorDataset(torch.randn(1000, 1000))
training_dataloader = DataLoader(training_dataset, batch_size=32, num_workers=4)
training_dataset[0][0].shape

torch.Size([1000])

In [10]:
for i_batch, batch in enumerate(training_dataloader):
    x = batch[0]
    outs = mutAE(x)
    print(x.shape, outs.shape)
    if i_batch==3:
        break

torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])


### Validation set

In [11]:
validation_dataset = dataset.TensorDataset(torch.randn(100, 1000))
validation_dataloader = DataLoader(validation_dataset, batch_size=32, num_workers=4)
validation_dataset[0][0].shape

torch.Size([1000])

In [12]:
for i_batch, batch in enumerate(validation_dataloader):
    x = batch[0]
    outs = mutAE(x)
    print(x.shape, outs.shape)
    if i_batch==3:
        break

torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([4, 1000]) torch.Size([4, 1000])


## Training with PyTorch Lightning

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [14]:
logger = TensorBoardLogger("random_test", name="VanillaAE")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [19]:
trainer.fit(model=mutAE, train_dataloader=validation_dataloader, val_dataloaders=validation_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name    | Type                 | Params
-------------------------------------------------
0 | encoder | FullyConnectedModule | 505 K 
1 | decoder | FullyConnectedModule | 506 K 
-------------------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.048     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [26]:
trainer.predict(model=mutAE, dataloaders=validation_dataloader)[0].shape

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 4it [00:00, ?it/s]

torch.Size([32, 10])

## Instantiate DRPredICT architecture: MultiEncoder

In [27]:
omic_types = ["mutation", "expression", "cn"]
in_dims = [1000, 1000, 1000]
out_dims = [6, 6, 6]
h_dims = [[500], [500], [500]]
encoder_args = [{"activation": "sigmoid"}, {"activation": "relu"}, {"activation": "sigmoid"}]
fcn_args = {"dropout_rate": 0.2, "output_dim": 1}

In [28]:
from MultiEncoder import MultiEncoder

In [29]:
drpredict = MultiEncoder(omics=omic_types, input_dims=in_dims, output_dims=out_dims, hidden_dims=h_dims, encoder_kwargs=encoder_args, fcn_kwargs=fcn_args)

## Load data

In [8]:
from torch.utils.data import dataset
from torch.utils.data import DataLoader

In [30]:
training_dataset = dataset.TensorDataset(torch.randn(1000, 1000), torch.randn(1000, 1000), torch.randn(1000, 1000), torch.randn(1000))
training_dataloader = DataLoader(training_dataset, batch_size=32, num_workers=4)
training_dataset[0][0].shape, training_dataset[0][1].shape, training_dataset[0][2].shape, training_dataset[0][3]

(torch.Size([1000]), torch.Size([1000]), torch.Size([1000]), tensor(0.0843))

In [33]:
validation_dataset = dataset.TensorDataset(torch.randn(100, 1000), torch.randn(100, 1000), torch.randn(100, 1000), torch.randn(100))
validation_dataloader = DataLoader(validation_dataset, batch_size=32, num_workers=4)
validation_dataset[0][0].shape, validation_dataset[0][1].shape, validation_dataset[0][2].shape, validation_dataset[0][3]

(torch.Size([1000]), torch.Size([1000]), torch.Size([1000]), tensor(-0.6169))

In [34]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [35]:
logger = TensorBoardLogger("random_test", name="MultiEncoder")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [37]:
trainer.fit(model=drpredict, train_dataloader=validation_dataloader, val_dataloaders=validation_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name     | Type                 | Params
--------------------------------------------------
0 | encoders | ModuleDict           | 1.5 M 
1 | fcn      | FullyConnectedModule | 19    
--------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.042     Total estimated model params size (MB)
/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory random_test/MultiEncoder/version_0/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [26]:
trainer.predict(model=mutAE, dataloaders=validation_dataloader)[0].shape

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 4it [00:00, ?it/s]

torch.Size([32, 10])

# MultiomicDataModule

## Instantiate DRPredICT architecture: VanillaAE

In [27]:
from VanillaAE import VanillaAE

In [28]:
encoder_args = {"activation": "sigmoid"}
decoder_args = {"dropout_rate": 0.2}

In [29]:
mutAE = VanillaAE("expression", 2000, 10, hidden_dims=[500], encoder_kwargs=encoder_args, decoder_kwargs=decoder_args)

In [30]:
x = torch.randn(10, 2000)
out = mutAE(x)
out.shape

torch.Size([10, 2000])

## Load data

In [8]:
from MultiomicDataModule import MultiomicDataModule

In [31]:
MULTIOMIC_DATA = "../data/training_top2000"

In [32]:
multiomic_datamodule = MultiomicDataModule(file_ext=MULTIOMIC_DATA, batch_size=512, num_workers=1, split=0.9, dataset_kwargs=dict(mutation=False, drug=False, expression=True, cn=False))

In [33]:
multiomic_datamodule.setup()

In [34]:
for i_batch, batch in enumerate(multiomic_datamodule.train_dataloader()):
    x = batch['expression']
    outs = mutAE(x)
    print(x.shape, outs.shape)
    if i_batch==3:
        break

torch.Size([512, 2000]) torch.Size([512, 2000])
torch.Size([124, 2000]) torch.Size([124, 2000])


In [35]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [36]:
logger = TensorBoardLogger("expression_test", name="VanillaAE")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [37]:
trainer.fit(model=mutAE, datamodule=multiomic_datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: expression_test/VanillaAE
Set SLURM handle signals.

  | Name    | Type                 | Params
-------------------------------------------------
0 | encoder | FullyConnectedModule | 1.0 M 
1 | decoder | FullyConnectedModule | 1.0 M 
-------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.052     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 42. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Instantiate DRPredICT architecture: MultiEncoder

In [2]:
omic_types = ["mutation", "expression"]
in_dims = [2000, 2000]
out_dims = [6, 6]
h_dims = [[500], [500]]
encoder_args = [{"activation": "sigmoid"}, {"activation": "relu"}]
fcn_args = {"dropout_rate": 0.2, "output_dim": 1}

In [3]:
from MultiEncoder import MultiEncoder

In [4]:
drpredict = MultiEncoder(omics=omic_types, input_dims=in_dims, output_dims=out_dims, hidden_dims=h_dims, encoder_kwargs=encoder_args, fcn_kwargs=fcn_args)

## Load data

In [99]:
MULTIOMIC_DATA = "../data/training_top2000"

In [104]:
from MultiomicDataModule import MultiomicDataModule

In [120]:
multiomic_datamodule = MultiomicDataModule(file_ext=MULTIOMIC_DATA, batch_size=32, num_workers=0, split=0.9,  dataset_kwargs=dict(drug=True, mutation=True, expression=True, cn=True))

In [121]:
multiomic_datamodule.setup()

In [122]:
for i_batch, batch in enumerate(multiomic_datamodule.test_dataloader()):
    print(batch["mutation"].shape, batch["expression"].shape, batch["cn"].shape, batch["auc"].shape)
    if i_batch==3:
        break

torch.Size([32, 2000]) torch.Size([32, 2000]) torch.Size([32, 2000]) torch.Size([32, 1])
torch.Size([32, 2000]) torch.Size([32, 2000]) torch.Size([32, 2000]) torch.Size([32, 1])
torch.Size([32, 2000]) torch.Size([32, 2000]) torch.Size([32, 2000]) torch.Size([32, 1])
torch.Size([32, 2000]) torch.Size([32, 2000]) torch.Size([32, 2000]) torch.Size([32, 1])


In [123]:
drpredict(batch).shape, batch["auc"].shape

(torch.Size([32, 1]), torch.Size([32, 1]))

In [124]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [125]:
logger = TensorBoardLogger("tri_test", name="MultiEncoder")
trainer = pl.Trainer(gpus="1", max_epochs=10, logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [126]:
trainer.fit(model=drpredict, datamodule=multiomic_datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name     | Type                 | Params
--------------------------------------------------
0 | encoders | ModuleDict           | 2.0 M 
1 | fcn      | FullyConnectedModule | 13    
--------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.028     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


# References